In [1]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import xgboost as xg
from sklearn.svm import SVR
from sklearn import preprocessing
from datetime import datetime as dt
from sklearn.model_selection import train_test_split, GroupKFold, cross_val_score, GridSearchCV
from sklearn.metrics import mean_absolute_error
from functools import partial
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, space_eval
from hyperopt.pyll import scope as ho_scope

In [2]:
# Importar la data
data = pd.read_csv('data.csv')

In [3]:
x = data.drop('actual_productivity', axis=1)
y = data['actual_productivity']

In [4]:
scaler = preprocessing.StandardScaler().fit(x)
x_scaled = scaler.transform(x)

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=127, random_state=10)

### Modelo simple

In [6]:
# Definir el modelo SVC
model_svm = SVR()

In [7]:
# Los parámetros que el modelo utiliza son

print('Parámetros el modelo:\n')
print(model_svm.get_params())

Parámetros el modelo:

{'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'degree': 3, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': False}


In [8]:
start = dt.now()
# Ajustar el modelo al conjunto de datos de entrenamiento
model_svm.fit(x_train, y_train)
end = dt.now()

In [9]:
# Tiempo que se requiere para ajustar el modelo
(end - start).seconds

0

In [10]:
# Valores predichos por el modelo
y_pred_train = model_svm.predict(x_train)
y_pred_test = model_svm.predict(x_test)

# Calcular el error para el conjunto de entrenamiento y prueba
error_train = mean_absolute_error(y_train, y_pred_train)
error_test = mean_absolute_error(y_test, y_pred_test)

In [11]:
# Se observa al aplicar un modelo SVM con sus parámetros por defecto
# el MAE para el conjunto de entrenamiento es de 0.08 y en el conjunto de pruebas es de 0.0876

print('MAE del conjunto de entrenamiento: {:.7f}'.format(error_train))
print('MAE del conjunto de prueba: {:.7f}'.format(error_test))

MAE del conjunto de entrenamiento: 0.0823429
MAE del conjunto de prueba: 0.0875906


### Modelo con ajuste de hiper parámetros

#### Grid search

In [73]:
# Crear la grilla de parámetros
param_grid = {
    'C': [1,5,10,50,100,250]
    , 'kernel': ['rbf']
    , 'gamma': [1,5,10, 20, 50, 100]
}

In [74]:
# Definir el modelo SVC
model_svm = SVR()

In [75]:
grid_search = GridSearchCV(estimator = model_svm
             , param_grid = param_grid
             , cv = 9
             ,verbose=1
             , n_jobs = -1)

In [76]:
start = dt.now()
grid_search.fit(x_train, y_train)
end = dt.now()

Fitting 9 folds for each of 36 candidates, totalling 324 fits


In [77]:
# Tiempo que se requiere para ajustar el modelo
(end - start).seconds

23

In [78]:
grid_search.best_params_

{'C': 1, 'gamma': 1, 'kernel': 'rbf'}

In [79]:
best_grid = grid_search.best_estimator_

In [80]:
joblib.dump(best_grid, 'resultados/smv_grid.pkl')

['resultados/smv_grid.pkl']

In [81]:
# Valores predichos por el modelo
y_pred_train_grid = best_grid.predict(x_train)
y_pred_test_grid = best_grid.predict(x_test)

# Calcular el error para el conjunto de entrenamiento y prueba
error_train_grid = mean_absolute_error(y_train, y_pred_train_grid)
error_test_grid = mean_absolute_error(y_test, y_pred_test_grid)

In [82]:
# Se observa al aplicar un modelo SVM con un ajuste de hiper parámetros
# el MAE para el conjunto de entrenamiento es de 0.0421 y en el conjunto de pruebas es de 0.065

print('MAE del conjunto de entrenamiento: {:.7f}'.format(error_train_grid))
print('MAE del conjunto de prueba: {:.7f}'.format(error_test_grid))

MAE del conjunto de entrenamiento: 0.0801134
MAE del conjunto de prueba: 0.1139265


#### Optimización bayesiana

In [57]:
def optimize(search_space):
    model = SVR(**search_space)
    scores = cross_val_score(model, X=x_train, y=y_train, cv=10, scoring='neg_mean_absolute_error').mean()
    
    return {'loss':scores, 'status': STATUS_OK, 'model': model}

In [60]:
search_space = {
    'C': ho_scope.int(hp.quniform('C', 1, 300, 50.1))
    ,'kernel': hp.choice('kernel', ['rbf'])
    ,'gamma': ho_scope.int(hp.quniform('gamma', 1, 100, 0.1))
}

optimization_function = partial(
    optimize
)

In [61]:
trials = Trials()
start = dt.now()

result = fmin(
    fn = optimization_function
    , space = search_space
    , max_evals = 200
    , trials=trials
    , algo = tpe.suggest
)

end = dt.now()

  2%|▉                                              | 4/200 [00:02<01:50,  1.78trial/s, best loss: -0.1403559269154754]

C:\Users\s6229429\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\s6229429\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\s6229429\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\Users\s6229429\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 315, in _dense_fit
    ) = libsvm.fit(
  File "sklearn\svm\_

  4%|██                                             | 9/200 [00:06<02:24,  1.32trial/s, best loss: -0.1411158768183627]

C:\Users\s6229429\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\s6229429\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\s6229429\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\Users\s6229429\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 315, in _dense_fit
    ) = libsvm.fit(
  File "sklearn\svm\_

 14%|██████▌                                      | 29/200 [00:22<02:15,  1.26trial/s, best loss: -0.14122454614949062]

...

C:\Users\s6229429\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\s6229429\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\s6229429\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\Users\s6229429\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 315, in _dense_fit
    ) = libsvm.fit(
  File "sklearn\svm\_

 98%|███████████████████████████████████████████ | 196/200 [01:16<00:02,  1.99trial/s, best loss: -0.14122454614949062]

C:\Users\s6229429\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\s6229429\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\s6229429\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\Users\s6229429\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 315, in _dense_fit
    ) = libsvm.fit(
  File "sklearn\svm\_

100%|████████████████████████████████████████████| 200/200 [01:17<00:00,  2.59trial/s, best loss: -0.14122454614949062]


C:\Users\s6229429\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\s6229429\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\s6229429\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\Users\s6229429\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 315, in _dense_fit
    ) = libsvm.fit(
  File "sklearn\svm\_

In [62]:
# Tiempo que se requiere para ajustar el modelo
(end - start).seconds

77

In [63]:
print(space_eval(search_space, result))

{'C': 300, 'gamma': 100, 'kernel': 'rbf'}


In [64]:
best_model_bo = trials.best_trial['result']['model']

In [65]:
joblib.dump(best_model_bo, 'resultados/svm_bo.pkl')

['resultados/svm_bo.pkl']

In [66]:
best_model_bo.fit(x_train, y_train)

SVR(C=300, gamma=100)

In [67]:
# Valores predichos por el modelo
y_pred_train_bo = best_model_bo.predict(x_train)
y_pred_test_bo = best_model_bo.predict(x_test)

# Calcular el error para el conjunto de entrenamiento y prueba
error_train_bo = mean_absolute_error(y_train, y_pred_train_bo)
error_test_bo = mean_absolute_error(y_test, y_pred_test_bo)

print('MAE del conjunto de entrenamiento: {:.7f}'.format(error_train_bo))
print('MAE del conjunto de prueba: {:.7f}'.format(error_test_bo))

MAE del conjunto de entrenamiento: 0.0802466
MAE del conjunto de prueba: 0.1317755


In [68]:
# trials.losses()

In [69]:
joblib.dump(trials, 'resultados/svm_bo_trials.pkl')

['resultados/svm_bo_trials.pkl']